In [ ]:
import os
import openai
from dotenv import load_dotenv
from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import load_index_from_storage
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


documents = SimpleDirectoryReader(input_files = ["./testdocument/introduction-to-natural-language-processing.pdf"]).load_data()
document = Document(text="\n\n".join([doc.text for doc in documents]))

def get_sentence_window_index(documents, index_dir, sentence_window_size=3):
    Node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_sentence",
    )

    Settings.llm = OpenAI()
    Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")  # Change model if neededs
    Settings.node_parser = Node_parser

    if not os.path.exists(index_dir):
        sentence_index = VectorStoreIndex.from_documents([document])
        sentence_index.storage_context.persist(persist_dir=index_dir)
        
    else:
        sentence_index = load_index_from_storage(StorageContext.from_defaults(persist_dir=index_dir))
    return sentence_index

def get_sentence_window_engine(sentence_index):
    
    postprocessor = MetadataReplacementPostProcessor(target_metadata_key="window",)
    rerank = SentenceTransformerRerank(top_n=2, model="BAAI/bge-reranker-base") 
    sentence_window_engine = sentence_index.as_query_engine(similarity_top_k=6, node_postprocessors=[postprocessor, rerank])
    
    return sentence_window_engine

In [2]:
index_dir = "./sentence_index_2"
sw_index_2 = get_sentence_window_index(documents, index_dir, sentence_window_size=3)
sw_engine_2 = get_sentence_window_engine(sw_index_2)


In [3]:
window_response_2 = sw_engine_2.query(
    "How are corpora utilized in grammatical research, and what advancements have been made with the use of computational tools?"
)
window_response_2.response

'Corpora are used in grammatical research to provide data for analysis and study. Advancements in computational tools have enabled researchers to conduct quantitative studies on various aspects of grammar, such as the frequency of different clause types in English. These tools have also facilitated the testing of predictions made by formal grammars developed within the generative school of linguistics. Additionally, parsed corpora and tools for analysis have made it possible to carry out more detailed and quantitative studies on grammatical structures and patterns within languages.'

In [4]:
from trulens_eval import Tru
from trulens_eval import TruLlama
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
from trulens_eval.app import App
import numpy as np

tru = Tru()

tru.reset_database()

# Initialize provider class
provider = OpenAI()

# Select context to be used in feedback. The location of context is app specific.
def get_evaluation_response(rag_engine, app_id, eval_questions):
    
    context = App.select_context(rag_engine)

    # Define a groundedness feedback function
    f_groundedness = (
        Feedback(provider.groundedness_measure_with_cot_reasons)
        .on(context.collect())  # Collect context chunks into a list
        .on_output()
    )

    # Question/answer relevance between overall question and answer.
    f_answer_relevance = (
        Feedback(provider.relevance)
        .on_input_output()
    )

    # Question/statement relevance between question and each context chunk.
    f_context_relevance = (
        Feedback(provider.context_relevance_with_cot_reasons)
        .on_input()
        .on(context)
        .aggregate(np.mean)
    )

    # Initialize the recorder
    tru_query_engine_recorder = TruLlama(
        rag_engine,
        app_id= app_id,
        feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])
    
    for question in eval_questions:
        with tru_query_engine_recorder as recording:
            response = rag_engine.query(question)
    records = recording.get()
    
    return records
    

/var/folders/bs/gpcpw2zj6r50ps5xm0ld7fj40000gn/T/ipykernel_34833/417823743.py:1: DeprecationWarning: The `trulens_eval` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval import Tru
/var/folders/bs/gpcpw2zj6r50ps5xm0ld7fj40000gn/T/ipykernel_34833/417823743.py:3: DeprecationWarning: The `trulens_eval.feedback` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval.feedback.provider import OpenAI
/var/folders/bs/gpcpw2zj6r50ps5xm0ld7fj40000gn/T/ipykernel_34833/417823743.py:3: DeprecationWarning: The `trulens_eval.feedback.provider` module is deprecated. See https://www.trulens.org/component_guides/other/trulens_eval_migration/ for instructions on migrating to `trulens.*` modules.
  from trulens_eval.feedback.provider import OpenAI
/var/folders/bs/gpcpw2zj6

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [5]:
eval_questions = [
    "Who introduced the notions of finite-state machines and context-free grammar (CFG) to linguistics?",
    "How did linguists test formal rules of grammar according to Chomsky’s approach?",
    "What has contributed to making the vision of computers understanding ordinary language and holding conversations with humans more plausible in the 21st century?"]

records = get_evaluation_response(
    sw_engine_2,
    app_id='sentence window engine 3',
    eval_questions = eval_questions
)

display(records)
tru.run_dashboard()

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_b

Record(record_id='record_hash_d26cc13c452943fe4bb7641ef60dc32a', app_id='app_hash_6f6e599e814916b1518e6a8df84a7460', cost=Cost(n_requests=1, n_successful_requests=1, n_completion_requests=1, n_classification_requests=0, n_classes=0, n_embedding_requests=0, n_embeddings=0, n_tokens=935, n_stream_chunks=0, n_prompt_tokens=908, n_completion_tokens=27, n_cortex_guardrails_tokens=0, cost=0.0014160000000000002, cost_currency='USD'), perf=Perf(start_time=datetime.datetime(2025, 3, 17, 21, 14, 29, 64113), end_time=datetime.datetime(2025, 3, 17, 21, 14, 33, 417540)), ts=datetime.datetime(2025, 3, 17, 21, 14, 33, 420476), tags='-', meta=None, main_input='What has contributed to making the vision of computers understanding ordinary language and holding conversations with humans more plausible in the 21st century?', main_output='Artificial intelligence techniques combined with the scientific study of language have emerged from universities and research laboratories, informing various industrial an

Starting dashboard ...


/var/folders/bs/gpcpw2zj6r50ps5xm0ld7fj40000gn/T/ipykernel_34833/1791174358.py:13: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:57820 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()